In [5]:
import math
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt


# Technical indicators
from ta.momentum import stochrsi,rsi
from ta.trend import ema_indicator, macd_diff, vortex_indicator_neg, vortex_indicator_pos, adx, cci, sma_indicator
from ta.volatility import bollinger_hband, bollinger_lband, average_true_range
from ta.volume import volume_weighted_average_price, ease_of_movement
# Technical indicators
from modules.Utils.indicators import addIndicators, computeSuperTrend
from modules.Utils.utils import loadFromDB
import warnings 
warnings.filterwarnings('ignore')

from datetime import datetime

# WITH NEWS METHODS

In [ ]:
# Technical indicators
from ta.trend import ema_indicator, trix, dpo
from ta.momentum import stochrsi
from ta.volatility import average_true_range
from modules.Utils.utils import loadFromDB, computeFutureLinearRegression, strategyTester
from modules.Utils.indicators import computeLaggingLinearRegression, computeTrixIndicator, addIndicators, computeRSI_VWAP, computeMASlope, computeSuperTrend

import pandas as pd

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
!ls ../backtest_tools/database/database/KuCoin/1d/

In [ ]:
SYMBOL= 'LINK'
df = loadFromDB(f'../backtest_tools/database/database/KuCoin/1h/{SYMBOL}-USDT.csv')
df.head()

In [ ]:
df['EMA']=ema_indicator(close=df['Close'], window=130)
df['STOCH_RSI'] = stochrsi(close=df['Close'], window=14, smooth1=3, smooth2=3)
df = computeSuperTrend(df)
import numpy as np

df_day = loadFromDB(f'../backtest_tools/database/database/KuCoin/1d/{SYMBOL}-USDT.csv')
df_day = computeSuperTrend(df_day)
df_day.dropna(inplace=True)
df_day.tail()
def addST(x):
    try:
        return df_day.loc[pd.to_datetime(x.name).date().strftime("%Y-%m-%d")].ST_Direction.values[0]
    except:
        return np.nan
    
df['ST_day'] = df.apply(addST,axis=1)
df.dropna(inplace=True)
df.head()

In [ ]:
def buyCondition(row:pd.Series, previous_row:pd.Series=None)->bool:
    return True if (row['STOCH_RSI']<0.8 and
                    row['ST_Direction']==True and
                    row['ST_day']==True and
                    row['Close']>row['EMA']) else False

def sellCondition(row:pd.Series, previous_row:pd.Series=None)->bool:
    return True if row['ST_Direction']==False and row['STOCH_RSI']>0.2 else False

strategyTester(df,buyCondition,sellCondition,equity=1000,)

In [ ]:
#results=[]
#for j in range(8,20,2):
#    for i in range(50,150,10):
#        SYMBOL= 'AVAX'
#        df = loadFromDB(f'../backtest_tools/database/database/KuCoin/1h/{SYMBOL}-USDT.csv')
#        df['EMA']=ema_indicator(close=df['Close'], window=i)
#        df['STOCH_RSI'] = stochrsi(close=df['Close'], window=j, smooth1=3, smooth2=3)
#        df = computeSuperTrend(df)
#        def buyCondition(row:pd.Series, previous_row:pd.Series=None)->bool:
#            return True if (row['STOCH_RSI']<0.8 and row['ST_Direction']==True and row['Close']>row['EMA']) else False
#
#        def sellCondition(row:pd.Series, previous_row:pd.Series=None)->bool:
#            return True if row['ST_Direction']==False and row['STOCH_RSI']>0.2 else False
#
#        coins = strategyTester(df,buyCondition,sellCondition,equity=1000,optimization_process=True)
#        result = {'EMA':i,'STOCH_RSI':j,'coins':coins}
#        print(result)
#        results.append(result)